# Amomaly Detection - One CLass


Part IV

# Acheviement :

|Model|Precision|Recall|F1|
|-----|---------|------|--|
|OneClassSVM|0.82|0.71|0.70|
|Isolation Forest|0.31|0.51|0.39|
|local outlier factor|0.71|0.71|0.70|

# Summary : 
One of the usefull methods in detecting abnormal behavious us anamoly detection where just one type of class going to be implemnted and fed to the classifer during the training .In the testing part the both classes normal and abnormal will be added to the classifer 

### Machine Learning Algorithms

to test this methods follwoing algorithms will be implemnted :

1. OneClassSVM
2. Local Outlier Factor
3. Isolation Forest 
4. Neaural network method-Autoencoder with its varitions

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import glob
import os


from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope

In [119]:
path = '/media/upm/New Volume/project/0'

In [120]:
# join all csv files avilable in the base path 
'''
csvs = glob.glob(os.path.join(path, '*.csv'))

df = pd.concat((pd.read_csv(f) for f in csvs)) # and concatenate them to one data frame using pandas and change the data types into numeric types
'''

"\ncsvs = glob.glob(os.path.join(path, '*.csv'))\n\ndf = pd.concat((pd.read_csv(f) for f in csvs)) # and concatenate them to one data frame using pandas and change the data types into numeric types\n"

In [143]:
df = pd.read_csv('/media/upm/New Volume/project/0/final.csv')

In [2]:
df = pd.read_csv('final1.csv')

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,protocol,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_mean,fwd_pkt_len_std,...,psh_flag_cnt,ack_flag_cnt,urg_flag_cnt,down_up_ratio,init_fwd_win_byts,init_bwd_win_byts,fwd_seg_size_min,active_mean,idle_mean,label_cat
0,0,0,6,141385.0,9,7,553,3773,61.444443,87.534440,...,1,0,0,0,8192.0,119.000000,20,0.0,0.0,0
1,1,1,6,281.0,2,1,38,0,19.000000,26.870058,...,0,1,0,0,123.0,0.000000,20,0.0,0.0,0
2,2,2,6,279824.0,11,15,1086,10527,98.727270,129.392500,...,1,0,0,1,8192.0,1047.000000,20,0.0,0.0,0
3,3,3,6,132.0,2,0,0,0,0.000000,0.000000,...,0,1,0,0,256.0,17682.268468,20,0.0,0.0,0
4,4,4,6,274016.0,9,13,1285,6141,142.777770,183.887730,...,1,0,0,1,8192.0,1047.000000,20,0.0,0.0,0


In [54]:
df = df.iloc[:100000, 2:]

In [55]:
df['B_cat'] = np.where(df['label_cat'],  -1 , 1) 

In [56]:
df['label_cat'].value_counts()

1    80182
0    19818
Name: label_cat, dtype: int64

In [57]:
df['label_cat'] = (df.label_cat != 0).astype('int')

In [58]:
df['B_cat'].value_counts()

-1    80182
 1    19818
Name: B_cat, dtype: int64

In [59]:
B = df.loc[df["B_cat"]  == 1 ]

In [60]:
B1 = B

In [61]:
mal = df.loc[df["B_cat"]  == -1 ]

In [64]:
mal1 = mal

In [65]:
B1['B_cat'].value_counts()

1    19818
Name: B_cat, dtype: int64

## split and train the algo on the benign traffic only 

In [66]:
X = B1.drop(['label_cat' , 'B_cat'], axis = 1)

In [67]:
y = B1['B_cat']

In [68]:
X.shape

(19818, 29)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y ,test_size=0.2 , random_state = 101 )

In [70]:
X_test.shape

(3964, 29)

## lets create test set of both classes 

In [71]:
mal1 = mal1.iloc[:3000,:]

In [72]:
mal11 = mal1['B_cat']

In [73]:
y_test.shape

(3964,)

In [74]:
new_x_test = pd.concat([X_test, mal1])

In [75]:
new_x_test.shape

(6964, 31)

In [76]:
new_x_test = new_x_test.drop(columns = ['label_cat', 'B_cat'] )

In [77]:
new_x_test.shape

(6964, 29)

In [78]:
new_y_test = pd.concat( [y_test, mal11] ) 

In [79]:
new_y_test.value_counts()

 1    3964
-1    3000
Name: B_cat, dtype: int64

# OneClassSVM

In [80]:
# define outlier detection model
model1 = OneClassSVM(gamma='scale' , kernel = 'rbf')

# fit on majority class
model1.fit(X_train , y_train)

OneClassSVM()

In [81]:
# detect outliers in the test set
yhat = model1.predict(new_x_test)

In [82]:
print("Accuracy :", metrics.accuracy_score(new_y_test , yhat))

Accuracy : 0.7110855829982768


In [83]:
print(classification_report(new_y_test , yhat))

              precision    recall  f1-score   support

          -1       0.60      1.00      0.75      3000
           1       1.00      0.49      0.66      3964

    accuracy                           0.71      6964
   macro avg       0.80      0.75      0.70      6964
weighted avg       0.82      0.71      0.70      6964



# Isolation Forest

In [84]:
clf =IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1, verbose=0)

clf.fit(X_train , y_train)

IsolationForest(n_jobs=-1)

In [85]:
pred = clf.predict(new_x_test)

In [86]:
print("Accuracy :", metrics.accuracy_score(new_y_test , pred))

Accuracy : 0.5142159678345778


In [87]:
print(classification_report(new_y_test , pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      3000
           1       0.54      0.90      0.68      3964

    accuracy                           0.51      6964
   macro avg       0.27      0.45      0.34      6964
weighted avg       0.31      0.51      0.39      6964



# Local Outlier Factor 

In [106]:
LOF = LocalOutlierFactor(novelty = True )

In [107]:
LOF.fit(X_train , y_train)

LocalOutlierFactor(novelty=True)

In [108]:
y_pred = LOF.predict(new_x_test)

/home/upm/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


In [109]:
print("Accuracy :", metrics.accuracy_score(new_y_test , y_pred))

Accuracy : 0.7074956921309592


In [110]:
print(classification_report(new_y_test , y_pred))

              precision    recall  f1-score   support

          -1       0.74      0.50      0.60      3000
           1       0.70      0.86      0.77      3964

    accuracy                           0.71      6964
   macro avg       0.72      0.68      0.68      6964
weighted avg       0.71      0.71      0.70      6964

